In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
from glob import glob
import base64
from matplotlib import pyplot as plt
import cv2

import torch

from dataset.dataset_unet import mask2data
from utils.mask_functions import rle2mask

mask_data = mask2data()

%matplotlib inline

KeyError: 'EncodedPixels'

## 0. Util 函数-mask与RLE格式互转

In [ ]:
mask_data[6]

In [ ]:
plt.imshow(rle2mask(mask_data[6]['masks'][0], 1024, 1024))

## 1. 数据集函数

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class MyDataset(Dataset):
    def __init__(self, img_id_list, IMG_SIZE, mode='train', augmentation=False):
        """传参，定义参数
        1. 数据集列表，
            - 本地数据，文件名/图片名
            - API，图片ID
        2. 图片读取尺寸
        3. 训练模式or推理模式
        4. 是否做Data augmentation
        ...
        """
        pass
    
    def __getitem__(self, idx):
        """读取下一个样本
        1. 读取本地图片，或读API接口获取base64格式图片
        2. 预处理, 如变换图片尺寸
        3. 若训练集，读取Mask图片
        4. Data augmentation
        """
        pass

    def __len__(self):
        """定义样本个数
        """
        pass

In [ ]:
def prepare_trainset():
    """
    1. 切分数据集，训练集/验证集
    2. 定义MyDataset训练集、MyDataset验证集
    3. 定义Pytorch的DataLoader
    
    train_dl = DataLoader(
            train_dataset,
            batch_size=16,
            shuffle=True,
            #sampler=sampler,
            num_workers=8,
            drop_last=True
        )
    val_dl = DataLoader(
            val_dataset,
            batch_size=16,
            shuffle=False,
            #sampler=sampler,
            num_workers=8,
            drop_last=True
        )
    """
    pass

In [ ]:
from dataset.dataset_unet import prepare_trainset

train_dl, val_dl = prepare_trainset(BATCH_SIZE=16, NUM_WORKERS=8, SEED=2019, IMG_SIZE=256, debug=True)

device = 'cpu'#'cuda:0'
for i, (images, masks) in enumerate(train_dl):#一次读取一个Bath
    images = images.to(device=device, dtype=torch.float)
    masks = masks.to(device=device, dtype=torch.float)
    if i==0:
        break

images.size(), masks.size()

## 2. Utils函数
    - 训练日志
    - 训练checkpoint
    - GPU交互

In [ ]:
## 参考utils/utils.py, utils/gpu_utils.py

## 3. 分割的评估函数
    - [Dice](https://zh.wikipedia.org/wiki/Dice%E7%B3%BB%E6%95%B0)
    - [IOU](https://zh.wikipedia.org/wiki/%E9%9B%85%E5%8D%A1%E5%B0%94%E6%8C%87%E6%95%B0)

In [ ]:
from PIL import Image
Image.open('media/IOU.png')

In [ ]:
Image.open('media/DICE.png')

In [ ]:
def dice(gt, pred):
    
    threshold = 0.5
    pred = (pred>threshold).astype(np.float)
    
    intersect = (pred * gt).sum()
    union = (pred + gt).sum()
    
    return ((2.0*intersect) / union)

def iou(gt, pred):
    
    threshold = 0.5
    pred = (pred>threshold).astype(np.float)
    
    intersect = (pred * gt).sum()
    union = (pred + gt).sum()
    
    return (intersect / (union - intersect))

metric = dice

In [ ]:
backgroud = np.zeros((128, 128))

gt = np.zeros((128, 128))
gt[64:96, 64:96] = 1.0

pred0 = np.zeros((128, 128))
pred0[64:96, 64:96] = 1.0
print(metric(gt, pred0))

plt.imshow(backgroud, cmap=plt.cm.binary)
plt.imshow(gt, cmap=plt.cm.Blues, alpha=0.1)
plt.imshow(pred0, cmap=plt.cm.Reds, alpha=0.2)

In [ ]:
pred1 = np.zeros((128, 128))
pred1[10:30, 10:30] = 1.0
print(metric(gt, pred1))

plt.imshow(backgroud, cmap=plt.cm.binary)
plt.imshow(gt, cmap=plt.cm.Blues, alpha=0.1)
plt.imshow(pred1, cmap=plt.cm.Reds, alpha=0.2)

In [ ]:
pred2 = np.zeros((128, 128))
pred2[55:90, 55:90] = 1.0
print(metric(gt, pred2))

plt.imshow(backgroud, cmap=plt.cm.binary)
plt.imshow(gt, cmap=plt.cm.Blues, alpha=0.1)
plt.imshow(pred2, cmap=plt.cm.Reds, alpha=0.2)

In [ ]:
## 思考：GT是空的，预测也是空的，Metric应该等于1，即完全预测正确，如何改正metric函数？
gt = np.zeros((128, 128))
pred = np.zeros((128, 128))

print(metric(gt, pred))

plt.imshow(backgroud, cmap=plt.cm.binary)
plt.imshow(gt, cmap=plt.cm.Blues, alpha=0.1)
plt.imshow(pred, cmap=plt.cm.Reds, alpha=0.2)

## 4. 训练脚本
    - train_unet.py

In [ ]:
def run_training():
    """training pipline
    
    1. 读取network
        - 加载预训练模型
        - 定义训练全部层的参数/哪几层参数
        - 定义学习率/为每一层定义学习率
        - 定义优化函数optimizer、学习率变化方案scheduler
        - 
    2. 训练N_EPOCH次迭代，每一个迭代内：
        - 用DataLoader循环读取训练集上每一个batch数据（N个图片、N个mask）
        - 将N个图片传入network，输出模型最后一层的预测（sigmoid概率）
        - 计算这个batch上的loss、metric，并存下来
        - 反向传播，更新参数（.backward()）（是否梯度累加）
        - 计算所有batch上loss、metric的总体均值，代表这个EPOCH
        - 用DataLoader循环读取验证集上每一个batch数据，与以上操作相似，计算验证集上的loss、metric，用于决定哪一个EPOCH停止训练
        - 更新logging、保存checkpoint
    """
    pass

## 5. Unet介绍

In [ ]:
Image.open('media/unet.png')

In [ ]:
import sys
sys.path.append('../')

import torch

#from model.model_unet import UNetResNet34
from model.model_unet_vanilla import UNetResNet34

In [ ]:
net = UNetResNet34(debug=True)
device = 'cuda:0'#'cpu'
net = net.to(device=device)

In [ ]:
for k,v in net.named_parameters():
    print(k, '='*20, v.size())

In [ ]:
from dataset.dataset_unet import prepare_trainset

train_dl, val_dl = prepare_trainset(BATCH_SIZE=4, NUM_WORKERS=8, SEED=2019, IMG_SIZE=256, debug=True)

device = 'cuda:0'
for i, (images, masks) in enumerate(train_dl):#一次读取一个Bath
    images = images.to(device=device, dtype=torch.float)
    masks = masks.to(device=device, dtype=torch.float)
    if i==0:
        break

images.size(), masks.size()

In [ ]:
logit = net(images)

In [ ]:
_loss = net.criterion(logit, masks)
_loss

In [ ]:
_metric = net.metric(logit, masks)
_metric